In [2]:
from utils.binaries import *

08:17:27 [INFO] utils.binaries -- (<99999ms) -- import numpy as np
08:17:27 [INFO] utils.binaries -- (    +7ms) -- import uncertainties
08:17:27 [INFO] utils.binaries -- (    +7ms) -- import uncertainties
08:17:27 [INFO] utils.binaries -- (  +131ms) -- import pandas as pd
08:17:27 [INFO] utils.binaries -- (    +2ms) -- import binaries.tools as tools


In [7]:
fmt = {
    'wcd' : np.dtype([('timestamp', 'I'), 
                    #('t1_latch_bin', 'h'), 
                    ('baseline', ('H', 3)), 
                    ('traces', ('h', (3, 2048)))]),
    'ssd' : np.dtype([('timestamp', 'I'), 
                    #('t1_latch_bin', 'h'),
                    ('baseline', 'H'), 
                    ('trace', ('h', 2048))])
}

def read(file_path, f) -> np.ndarray :

    # buffer = bz2.BZ2File(file_path).read()        
    return np.fromfile(file_path, fmt[f])


with open('/cr/tempdata01/filip/UubRandoms/2023_03/converted/timestamps/Nadia.dat', 'w') as timestamps:
    for i in tools.ProgressBar(range(1434), newline=False):
        path_to_file = f'/cr/data02/AugerPrime/UUB/UubRandoms/2023_03/Nadia/randoms{i:04}.wcd'
        data = read(path_to_file, 'wcd')['timestamp']
        if len(data) == 0: continue

        timestamps.write(f"{path_to_file.split('/')[-1]} {int(data.min())} {int(data.max())}\n")



[1678833453 1678833453 1678833453 ... 1678833503 1678833503 1678833503]84.83 it/s
